In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import skimage.transform as trans
import sys

In [2]:
# import models
sys.path.insert(1, 'CNN/')
sys.path.insert(2, 'UNet/')
import model
import U_Net_model

In [3]:
cnn_weight = 'CNN/weights/weights.hdf5'
unet_weight = 'UNet/weights/u_netweights.hdf5'
image_path = 'test_1.jpg' 

In [4]:
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = image / 255
image = trans.resize(image,(256,256))

In [5]:
x = []
x.append(image)
x = np.array(x)
x = np.reshape(x, (1, 256, 256, 1))

In [6]:
cnn = model.cnn_bt(pretrained_weights=cnn_weight)
pd_cnn = cnn.predict(x)

In [7]:
if pd_cnn[0][0] > 0.5:
    print('Tumor detected.')
    org = cv2.imread(image_path)
    rows, cols, channels = org.shape     
    unet = U_Net_model.unet_bt(pretrained_weights=unet_weight)
    unet.compile(optimizer = Adam(), loss = 'binary_crossentropy')
    predicted = unet.predict(np.reshape(image, (1, 256, 256, 1)))
    predicted = predicted.astype(np.float64) * 255
    predicted = np.reshape(predicted, (256, 256))
    predicted = trans.resize(predicted, (rows,cols))
    predicted = predicted.astype(np.uint8)
    predicted = cv2.cvtColor(predicted, cv2.COLOR_GRAY2BGR)
   
    ret, mask = cv2.threshold(predicted, 120, 255, cv2.THRESH_BINARY)
    white_pixels = np.where((mask[:, :, 0] == 255) & 
                            (mask[:, :, 1] == 255) & 
                            (mask[:, :, 2] == 255))
    mask[white_pixels] = [255, 255, 0]
    add = cv2.addWeighted(org, 0.9, mask, 0.7, 0)   
    
    add = cv2.putText(add,
                     'Tumor detected.',
                     (int(rows/20),int(cols/15)),
                     cv2.FONT_HERSHEY_SIMPLEX,
                     cols/(rows+cols),
                     (255,255,255),
                     1,
                     cv2.LINE_AA)
    cv2.imshow('image', add)
    
    file_name = image_path.replace("/", "")
    file_name = file_name.replace(".jpg", "")
    cv2.imwrite('{}predicted.png'.format(file_name),
                add)   
else:
    print('no tumor detected.')
    org = cv2.imread(image_path)
    rows, cols, channels = org.shape     
    org = cv2.putText(org,
                     'No tumor detected.',
                     (int(rows/20),int(cols/15)),
                     cv2.FONT_HERSHEY_SIMPLEX,
                     cols/(rows+cols),
                     (255,255,255),
                     1,
                     cv2.LINE_AA)
    cv2.imshow('image', org)
    file_name = image_path.replace("/", "")
    file_name = file_name.replace(".jpg", "")
    cv2.imwrite('{}predicted.png'.format(file_name),
                org)
cv2.waitKey(0)
cv2.destroyAllWindows()

Tumor detected.
